In [1]:
import csv
import pyodbc
from datetime import datetime,timedelta

In [2]:
planningfile = r'c:\data\planning_data.csv'

In [3]:
extract_date = []
publisher_uri = []
publisher_label = []
organisation_uri = []
organisation_label = []
case_reference = []
case_url = []
case_date = []
service_type_uri = []
service_type_label = []
classification_uri = []
classification_label = []
case_ = []
location =[]
decision_target_date = []
status = []
decision_date = []
decision = []
decision_type = []
agent = []



In [4]:
def clean_date(d):
    try:
        date = datetime.strptime(d, '%d/%m/%Y')
    except:
        date = None
    return date

In [5]:
clean_date('05/03/2019')

datetime.datetime(2019, 3, 5, 0, 0)

In [7]:
with open (planningfile, encoding = 'latin') as f:
    reader = csv.DictReader(f)
    for row in reader:
        extract_date.append(clean_date(row['ExtractDate']))
        publisher_uri.append(row['PublisherURI'])
        publisher_label.append(row['PublisherLabel'])
        organisation_uri.append(row['OrganisationURI'])
        organisation_label.append(row['OrganisationLabel'])
        case_reference.append(row['CaseReference'])
        case_url.append(row['CaseURL'])
        case_date.append(clean_date(row['CaseDate']))
        service_type_uri.append(row['ServiceTypeURI'])
        service_type_label.append(row['ServiceTypeLabel'])
        classification_uri.append (row['ClassificationURI'])
        classification_label.append(row['ClassificationLabel'])
        case_.append(row['CaseText'])
        location.append(row['LocationText'])
        decision_target_date.append(clean_date(row['DecisionTargetDate']))
        status.append(row['Status'])
        decision_date.append(clean_date(row['DecisionDate']))
        decision.append(row['Decision'])
        decision_type.append(row['DecisionType'])
        agent.append(row['Agent'])

In [8]:
data = zip(extract_date, publisher_uri, publisher_label, organisation_uri, organisation_label, case_reference, case_url, case_date, service_type_uri, service_type_label, classification_uri, classification_label, case_, location, decision_target_date, status, decision_date, decision, decision_type, agent)

In [19]:
conn = pyodbc.connect('DSN=kubricksql;UID=de14;PWD=password')
cur = conn.cursor()


In [15]:
qapproved = 'insert into brigitte.approved(extract_date, publisher_uri, publisher_label, organisation_uri, organisation_label, case_reference, case_url, case_date, service_type_uri, service_type_label, classification_uri, classification_label, case_, location, decision_target_date, status, decision_date, decision, decision_type, agent) values (\
?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'
qrejected = 'insert into brigitte.rejected(extract_date, publisher_uri, publisher_label, organisation_uri, organisation_label, case_reference, case_url, case_date, service_type_uri, service_type_label, classification_uri, classification_label, case_, location, decision_target_date, status, decision_date, decision, decision_type, agent) values (\
?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'
qother = 'insert into brigitte.other(extract_date, publisher_uri, publisher_label, organisation_uri, organisation_label, case_reference, case_url, case_date, service_type_uri, service_type_label, classification_uri, classification_label, case_, location, decision_target_date, status, decision_date, decision, decision_type, agent) values (\
?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'

In [21]:
for d in data:
    if d[18] == 'Approve':
        try:
            cur.execute(qapproved,d)
            conn.commit
        except:
            conn.rollback()
    elif d[18] =='Refuse':
        try:
            cur.execute(qrejected,d)
            conn.commit
        except:
            conn.rollback()
    else:
        try:
            cur.execute(qother,d)
            conn.commit
        except:1
            conn.rollback()
    